In [2]:
import pandas as pd
import spark
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np


2024-10-26 16:21:59.331756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-26 16:21:59.332319: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-26 16:21:59.334876: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-26 16:21:59.342589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729952519.356840   34319 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729952519.36

In [3]:
#read the data

activitats = pd.read_csv('data/activitats.csv')
notes = pd.read_csv('data/notes.csv')
trameses = pd.read_csv('data/trameses.csv')

#show the columns

print(activitats.columns)

#show the first 5 rows

print(activitats.head())



Index(['activitat_id', 'activitat', 'aula_id', 'startdate', 'duedate',
       'grade'],
      dtype='object')
   activitat_id                                activitat  aula_id  startdate  \
0             3            Problema 1.1 - Hello world!!!       87          0   
1             4         Problema 1.2 - Hello world!!! ++       87          0   
2             5               Problema 3a.1: C�lcul edat       87          0   
3             6  Problema 3a.8: Conversi� d�lars a euros       87          0   
4             7         Problema 3a.11: Mitjana de notes       87          0   

   duedate  grade  
0        0    100  
1        0    100  
2        0    100  
3        0    100  
4        0    100  


Comencem treient totes les activitats que no tenen cap tramesa associada


In [4]:

# Filter activitats to keep only entries with an activitat_id present in trameses
activitats_filtered = activitats[activitats['activitat_id'].isin(trameses['activitat_id'].unique())]

# Save or display the filtered activitats DataFrame
activitats_filtered.to_csv('activitats_filtered.csv', index=False)
print(activitats_filtered.head())


   activitat_id                                activitat  aula_id  startdate  \
0             3            Problema 1.1 - Hello world!!!       87          0   
1             4         Problema 1.2 - Hello world!!! ++       87          0   
2             5               Problema 3a.1: C�lcul edat       87          0   
3             6  Problema 3a.8: Conversi� d�lars a euros       87          0   
4             7         Problema 3a.11: Mitjana de notes       87          0   

   duedate  grade  
0        0    100  
1        0    100  
2        0    100  
3        0    100  
4        0    100  


A continuació afegim una clau shared_id que s'encarrega de relacionar tasques identiques que es troben a cursos diferents les que no pertanyen a cap d'aquestses classes

In [5]:
# Create a dictionary to map each unique activitat to a unique ID
activitat_unique_id = {activitats_filtered: idx for idx, activitats_filtered in enumerate(activitats_filtered['activitat'].unique(), start=1)}

# Map the activitat values to the unique IDs in the pastas column
activitats_filtered['shared_id'] = activitats_filtered['activitat'].map(activitat_unique_id)

# Save or display the updated activitats DataFrame
activitats_filtered.to_csv('activitats_with_shared.csv', index=False)

/tmp/ipykernel_34319/3240484575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activitats_filtered['shared_id'] = activitats_filtered['activitat'].map(activitat_unique_id)


Per últim separem per les cuatre aules de les quals tenim a disposició notes d'exàmens, i esborrem del general les que no pertanyen a cap d'aquestes aules

In [6]:
# Define the aula_id values to filter
aula_ids = [87, 92, 143, 141]

activitats_filtered = activitats_filtered[activitats_filtered['aula_id'].isin(aula_ids)]

activitats_filtered.to_csv('activitats_with_shared.csv', index=False)
# Loop through each aula_id, filter, and save as a new CSV file
for aula_id in aula_ids:
    # Filter activitats for the current aula_id
    filtered_df = activitats_filtered[activitats_filtered['aula_id'] == aula_id]
    
    # Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(f'activitats_aula_{aula_id}.csv', index=False)

print("CSV files created for each specified aula_id.")


CSV files created for each specified aula_id.


Added lables for diferent types of exercises

In [10]:
keywords = {
        "(OPCIONAL)": 3,

    "Problema": 1,
    "Entrega Practica": 2,
    "Entrega Pràctica": 2,
    "Lliurament Practica": 2,
    "Lliurament Pr�ctica": 2,
    "Lliurament Pràctica": 2,
    "Pr�ctica": 2,
    "RECUPERACI�": 3,
    "Recuperació": 3,
    "RECUPERACIÓ": 3,
    "Final FP": 4,
    "Exercici": 5,
    # Add more keywords and labels as needed
}

# Function to check for keywords in each activitat entry and assign a label
def label_activity(activitat):
    for keyword, label in keywords.items():
        if keyword in activitat:
            return label
    return 0  # Default label if no keywords match

# Apply the function to create a new 'label' column
activitats_filtered['label'] = activitats_filtered['activitat'].apply(label_activity)
activitats_filtered.to_csv('activitats_with_shared.csv', index=False)


from mamba_ssm import Mamba

batch, length, dim = 2, 64, 16
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
y = model(x)
assert y.shape == x.shape